In [17]:
from dotenv import load_dotenv
import os

load_dotenv()  # reads .env and injects into environment
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# retrieve from environment
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


Docs:

https://blog.langchain.dev/semi-structured-multi-modal-rag/

https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector

Paper:

https://arxiv.org/abs/2312.06648

In [18]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs.extend(loader.load())

In [19]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | ChatOpenAI(model="gpt-3.5-turbo",max_retries=0)
    | StrOutputParser()
)

summaries = chain.batch(docs, {"max_concurrency": 5})

In [20]:
import uuid
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from pydantic import Field

# Custom Multi-Vector Retriever implementation
class MultiVectorRetriever(BaseRetriever):
    """Retriever that searches summaries but returns full documents."""
    
    vectorstore: Chroma = Field(...)
    docstore: dict = Field(...)
    id_key: str = Field(default="doc_id")
    search_kwargs: dict = Field(default_factory=lambda: {"k": 4})
    
    def _get_relevant_documents(
        self, 
        query: str, 
        *, 
        run_manager: CallbackManagerForRetrieverRun = None
    ):
        # Search for relevant summaries
        summary_docs = self.vectorstore.similarity_search(query, **self.search_kwargs)
        
        # Get doc_ids from summaries
        doc_ids = [doc.metadata.get(self.id_key) for doc in summary_docs if self.id_key in doc.metadata]
        
        # Return full documents from docstore
        full_docs = [self.docstore[doc_id] for doc_id in doc_ids if doc_id in self.docstore]
        return full_docs

# The vectorstore to use to index the summaries
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=OpenAIEmbeddings())

# Simple in-memory storage for parent documents
docstore = {}
id_key = "doc_id"

# Generate unique IDs for documents
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Store full documents
for doc_id, doc in zip(doc_ids, docs):
    docstore[doc_id] = doc

# Create summary documents with doc_ids in metadata
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

# Add summaries to vectorstore
vectorstore.add_documents(summary_docs)

# Create the retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key=id_key,
)

In [21]:
query = "Memory in agents"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]

Document(metadata={'doc_id': 'bea633f4-2eb7-45c1-8939-f7086f0d3b17'}, page_content='This document discusses the concept of building autonomous agents powered by Large Language Models (LLM). It covers key components of LLM-powered agents, such as planning, memory, and tool use. Proof-of-concept examples like AutoGPT and GPT-Engineer are provided to demonstrate the potential applications of LLM-powered agents. Challenges, such as finite context length, reliability of natural language interface, and long-term planning, are also discussed. The document provides a comprehensive overview of the capabilities and limitations of LLM-powered autonomous agents.')

In [22]:
# Use invoke (Runnable API) - BaseRetriever implements Runnable
# Temporarily set k=1 for this query
original_k = retriever.search_kwargs.get("k", 4)
retriever.search_kwargs["k"] = 1
retrieved_docs = retriever.invoke(query)
retriever.search_kwargs["k"] = original_k  # Reset to default

if retrieved_docs:
    print(retrieved_docs[0].page_content[0:500])
else:
    print("No documents retrieved")







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log

















|






Posts




Archive




Search




Tags




FAQ









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three:
